In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from IPython.display import display
from pathlib import Path

In [2]:
#Excel reading function
df_dict = pd.read_excel(r"C:\Users\mattp\Finance Project\Practical Analysis\All Assets Price Change.xlsx",  sheet_name = None)

df_AllAssets = df_dict['All Assets']
df_AllAssets = df_AllAssets.drop(["Unnamed: 0"], axis = 1)

del df_dict['All Assets']

display(df_dict.keys())

df_AllAssets.columns = df_AllAssets.columns.to_series().replace('Unnamed:\s\d+',np.nan,regex=True).ffill().values

dict_keys(['Stock- NVIDIA', 'Stock- Amazon', 'Stock- Apple', 'Stock- Microsoft', 'Stock- Tesla', 'Mutual Fund- FNF', 'Mutual Fund-FSTP', 'Bond- 13 Week TBill', 'Bond- 5YTBY', 'Bond- 10 YTBill Yield', 'Commodity- Copper', 'Commodity- Oil', 'Commodity- Gold', 'Commodity- Steel', 'Credit Data- SNP Futures', 'FX Pair- USD EUR', 'FX Pair- USD JPY', 'Index Fund- S&P'])

In [3]:
dict_weights = {}

for key in df_dict:
    df_common = df_dict[key]
    column_common = df_common["Weights"]
    column_common = column_common.dropna()
    column_common = column_common.iloc[-1:]
    column_common = column_common.reset_index(drop=True)
    dict_weights[key] = column_common


In [4]:
weights_list = [i for i in dict_weights.values()]
weights_array = np.array(weights_list)

In [5]:
dict_corrDfs = {}

for key in df_dict:
    df_common = df_dict[key]
    column_common = df_common["Log Returns"]
    column_common = column_common.dropna()
    column_common = column_common.iloc[-90:]
    column_common = column_common.reset_index()
    column_common = column_common.drop(["index"], axis = 1)
    dict_corrDfs[key] = column_common

In [6]:
df_log_returns = pd.concat(dict_corrDfs.values(), axis = 1, keys=dict_corrDfs)
df_log_returns.to_csv("Log returns.csv")

In [7]:
df_corr = df_log_returns.corr()
df_corr.to_excel("Correlation Matrix.xlsx")

In [8]:
portfolio_volatility = (np.sqrt(np.dot(weights_array.T, np.dot(df_log_returns.cov() * np.sqrt(365), weights_array))))[0,0]
display(portfolio_volatility)
portfolio_variance = np.dot(weights_array.T,np.dot(df_corr, weights_array))

0.03545236724660104

In [9]:
#Expected return of portfolio
exp_ret = np.sum((df_log_returns.mean() * weights_array.T[0])*np.sqrt(365))
print(exp_ret)


0.017337143477315017


In [10]:
#Sharpe ratio of portfolio
#used the 10 year treasury rate
risk_free_rate = 0.0534

sharpe_ratio = (exp_ret - risk_free_rate)/portfolio_volatility
print(sharpe_ratio)



-1.0172199862378013


In [11]:
list_dfNames = ["Stock- NVIDIA", "Stock- Amazon", "Stock- Apple", "Stock- Microsoft", "Stock- Tesla", "Mutual Fund- FNF", "Mutual Fund-FSTP", "Bond- 13 Week TBill", "Bond- 5YTBY", "Bond- 10 YTBill Yield", "Commodity- Copper", "Commodity- Oil", "Commodity- Gold", "Commodity- Steel", "Credit Data- SNP Futures", "FX Pair- USD EUR", "FX Pair- USD JPY", "Index Fund- S&P"]
df_Data_Concat = pd.concat(df_dict.values(), axis = 1, keys=list_dfNames[:])

In [12]:
writer = pd.ExcelWriter(r"C:\Users\mattp\Finance Project\Practical Analysis\Portfolio.xlsx")

df_AllAssets["Volatility of Portfolio"] = portfolio_volatility/100
df_AllAssets["Sharpe Ratio of Portfolio"] = sharpe_ratio
df_AllAssets.to_excel(writer, sheet_name="All Assets")

for key in df_dict:
    new_dataframe = df_Data_Concat[key]
    new_dataframe.to_excel(writer, sheet_name=key)

writer.close()